<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220226_sm_hardvoting_with_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full training

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 22.3 MB/s 
     |████████████████████████████████| 67 kB 3.0 MB/s 
     |████████████████████████████████| 134 kB 45.5 MB/s 
     |████████████████████████████████| 212 kB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.9 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 271 kB 45.6 MB/s 
     |████████████████████████████████| 144 kB 45.7 MB/s 
     |████████████████████████████████| 895 kB 38.2 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 6.8 MB 40.4 MB/s 
     |████████████████████████████████| 1.2 MB 32.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 2.4 MB/s 
     |████████████████████████████████| 149.9 

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [81]:
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli.sample(frac=1).reset_index(drop=True)
kakao_snli = kakao_snli[:50000]

PATH =  '/content/drive/MyDrive/KLUE'
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_train = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_train.json')
# train = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')

klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_train = klue_train[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)
klue_train.rename(columns = {'gold_label':'label'}, inplace=True)

train = pd.concat([kakao_dev,kakao_test, kakao_snli])
train.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)
train = pd.concat([train, klue_dev, klue_train], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train

,premise,hypothesis,label,index
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral,0
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction,1
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment,2
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral,3
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction,4
...,...,...,...,...
84518,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",개발 음식 활용 혜택만 주어진다.,contradiction,84518
84519,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",힐링푸드 전문가에게 혜택이 주어진다.,neutral,84519
84520,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",혜택에는 힐링푸드 전문가 양성 교육 참여가 있다.,entailment,84520
84521,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction,84521


In [7]:
from transformers import AutoTokenizer

checkpoint = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [9]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [10]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label

In [13]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [20]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=16, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=16, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220226/model' + str(fold), save_function=accelerator.save)

In [21]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['premise']),
  list(train['hypothesis']),
  return_tensors="pt",
  max_length=128, # Max_Length = 190
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = label_to_num(train['label'].values)

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

...

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

epoch 1 train acc 0.34210526315789475
epoch 1 train acc 0.34539473684210525
epoch 1 train acc 0.35855263157894735
epoch 1 train acc 0.34539473684210525
epoch 1 train acc 0.3157894736842105
epoch 1 train acc 0.35526315789473684
epoch 1 train acc 0.3059210526315789
epoch 1 train acc 0.35855263157894735
epoch 1 test acc 0.375
epoch 1 test acc 0.3
epoch 1 test acc 0.4125
epoch 1 test acc 0.325
epoch 1 test acc 0.2875
epoch 1 test acc 0.2875
epoch 1 test acc 0.3875
epoch 1 test acc 0.275
epoch 2 train acc 0.41118421052631576
epoch 2 train acc 0.375
epoch 2 train acc 0.41118421052631576
epoch 2 train acc 0.3881578947368421
epoch 2 train acc 0.40131578947368424
epoch 2 train acc 0.39473684210526316
epoch 2 train acc 0.3519736842105263
epoch 2 train acc 0.3881578947368421
epoch 2 test acc 0.5375
epoch 2 test acc 0.6125
epoch 2 test acc 0.575
epoch 2 test acc 0.625
epoch 2 test acc 0.5125
epoch 2 test acc 0.5375
epoch 2 test acc 0.6
epoch 2 test acc 0.5
epoch 3 train acc 0.625
epoch 3 train acc

In [22]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220226/model' + str(fold), num_labels=3)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)
  # model.load_state_dict(torch.load('/content/drive/MyDrive/KLUE_model/hj_model' + str(fold) + '.pt'))

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)

      output_pred.append(result)
      output_prob.append(prob)
    
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df = pd.DataFrame(answer, columns=['index', 'label'])
  
  # df = pd.DataFrame(output_prob)

  df.to_csv('/content/drive/MyDrive/220226/pred'+str(fold)+'.csv', index=False)

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

In [59]:
pred0 = pd.read_csv('/content/drive/MyDrive/KLUE/pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/KLUE/pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/KLUE/pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/KLUE/pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/KLUE/pred4.csv')

In [60]:
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

In [61]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']

In [62]:
df = test

In [63]:
df

,index,premise,hypothesis,label,label0,label1,label2,label3,label4
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,answer,contradiction,contradiction,contradiction,contradiction,contradiction
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,answer,neutral,neutral,neutral,neutral,neutral
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,answer,neutral,neutral,entailment,entailment,neutral
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,answer,contradiction,contradiction,contradiction,contradiction,contradiction
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,answer,entailment,contradiction,contradiction,contradiction,contradiction
...,...,...,...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,answer,neutral,neutral,neutral,neutral,neutral
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,answer,neutral,neutral,neutral,neutral,neutral
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,answer,neutral,neutral,neutral,neutral,neutral
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,answer,neutral,neutral,neutral,neutral,neutral


In [64]:
def label_count(df):
  num_neutral = 0
  num_contradiction = 0
  num_entailment = 0
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']
  for col in cols:
    if df[col] == 'neutral':
      num_neutral = num_neutral + 1
    if df[col] == 'contradiction':
      num_contradiction = num_contradiction + 1
    if df[col] == 'entailment':
      num_entailment = num_entailment + 1
  return [num_neutral, num_contradiction, num_entailment]

temp = df.apply(label_count, axis=1)
df['temp'] = temp
df['neutral'] = 0
df['contradiction'] = 0
df['entailment'] = 0
def list_to_num(list:list):
  return list[0]

df['neutral'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[1]

df['contradiction'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[2]

df['entailment'] = df['temp'].map(list_to_num)

def voting(df):
  cols = ['neutral', 'contradiction', 'entailment']
  for col in cols:
    if df[col] > 2:
      return col
  return 'neutral'

df['label'] = df.apply(voting, axis=1)

In [65]:
df['label'].value_counts()

neutral          685
contradiction    497
entailment       484
Name: label, dtype: int64

In [66]:
def make_label(list:list):
  if list == [3, 2, 0] :
    return 'contradiction'
  if list == [3, 0, 2]:
    return 'entailment'
  if list == [4, 1, 0]:
    return 'contradiction'
  if list == [4, 0, 1]:
    return 'entailment'
  else :
    return 'answer'

my_label = df['temp'].map(make_label)

In [67]:
df['my_label'] = my_label

In [68]:
my_answer = df[df['my_label'] != 'answer']

In [69]:
df.loc[my_answer.index, 'label'] = my_answer['my_label']

In [70]:
df['label'].value_counts()

contradiction    564
neutral          564
entailment       538
Name: label, dtype: int64

In [76]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [77]:
submission['label'] = df['label']

In [78]:
submission

,index,label
0,0,contradiction
1,1,neutral
2,2,entailment
3,3,contradiction
4,4,contradiction
...,...,...
1661,1661,neutral
1662,1662,neutral
1663,1663,neutral
1664,1664,neutral


In [79]:
submission.to_csv('/content/drive/MyDrive/KLUE/submission.csv', index=False)